In [24]:
# =========================================
# Housing Price Prediction Capstone Project
# =========================================
#
# **Author:** Krishnan Ramaswami  
# 
# 
# ## Summary
# End-to-end regression modeling for housing prices: data preprocessing, feature engineering, EDA, baseline & multiple models, 
# hyperparameter tuning, SVR with kernel trick, bias-variance analysis, and final evaluation visualizations.
#
#
# =========================================
# Import Libraries
# =========================================
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time
import warnings
import sys
import contextlib
import io
import matplotlib.colors as mcolors
from math import pi
from sklearn.exceptions import ConvergenceWarning
warnings.filterwarnings("ignore", category=ConvergenceWarning)
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split,RandomizedSearchCV, GridSearchCV, learning_curve
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.linear_model import LinearRegression, Ridge, SGDRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor, ExtraTreesRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from datetime import datetime
from IPython.display import display, HTML
from colorama import Fore, Style, init
init(autoreset=True)


# Inline plots
%matplotlib inline
sns.set(style="whitegrid")
plt.rcParams['figure.figsize'] = (10,6)

# ===============================
# 🏠 Capstone Project Header
# ===============================

def project_header(title, subtitle="", color="#1565C0", icon="🏠"):
    """
    Displays a professional project header at the start of the notebook.
    """
    display(HTML(
        f"<div style='background-color:#E3F2FD; border-left:8px solid {color}; "
        f"padding:16px; border-radius:8px; margin:10px 0;'>"
        f"<h2 style='color:{color}; margin:0;'>{icon} {title}</h2>"
        f"<p style='color:#333; font-size:15px; margin-top:6px;'>{subtitle}</p>"
        f"</div>"
    ))
def stage_message(title, color="#4CAF50", icon="✅"):
    """
    Displays a styled, color-coded message header for notebook progress updates.
    
    Parameters:
        title (str): Description of the current or completed stage.
        color (str): Hex code for text color (default green).
        icon (str): Emoji icon to visually represent the stage.
    """
    display(HTML(
        f"<div style='background-color:#F9F9F9; border-left:6px solid {color}; "
        f"padding:8px 12px; margin:6px 0; border-radius:6px;'>"
        f"<p style='font-size:15px; font-weight:bold; color:{color}; margin:0;'>"
        f"{icon} {title}</p></div>"
    ))

# =========================================
# Load and Inspect Dataset
# =========================================
project_header(
    "California Housing Price Prediction – Capstone Project",
    subtitle="A comprehensive machine learning pipeline for real-estate value estimation using the California Housing dataset.",
    color="#0D47A1",
    icon="📊"
)
housing = fetch_california_housing()

# Convert to DataFrame
housing_df = pd.DataFrame(housing.data,
                             columns=housing.feature_names)
housing_df['MedHouseValue'] = pd.Series(housing.target)
#display(housing_df.head())
#print(housing_df.info())
#print(housing_df.describe())
#print(housing_df.isnull().sum())

stage_message("Loaded dataset 'scikit-learn – California Housing Dataset'", color="#2196F3", icon="📘")

# =========================================
# Feature & Task Understanding
# =========================================
target = 'MedHouseValue'
predictors = [col for col in housing_df.columns if col != target]
numerical_features = housing_df[predictors].select_dtypes(include=['int64','float64']).columns.tolist()
categorical_features = housing_df[predictors].select_dtypes(include=['object','category']).columns.tolist()
print("Numerical Features:", numerical_features)
print("Categorical Features:", categorical_features)

plt.figure()
sns.heatmap(housing_df.corr(), annot=True, cmap='coolwarm', fmt=".2f")
plt.title("Correlation Heatmap")
plt.savefig("images/Correlation Heatmap.png")
plt.close()
stage_message("Feature & Task Understanding Completed", color="#673AB7", icon="🧭")
print("   ➤ Feature types identified: Numerical and Derived ratios analyzed.\n")

# =========================================
# Data Clean up and Preprocessing
# =========================================
X = housing_df[predictors]
y = housing_df[target]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale numerical features
scaler = StandardScaler()
X_train[numerical_features] = scaler.fit_transform(X_train[numerical_features])
X_test[numerical_features] = scaler.transform(X_test[numerical_features])

# Encode categorical features
if categorical_features:
    encoder = OneHotEncoder(drop='first', sparse=False)
    X_train_encoded = pd.DataFrame(encoder.fit_transform(X_train[categorical_features]),
                                   columns=encoder.get_feature_names_out(categorical_features),
                                   index=X_train.index)
    X_test_encoded = pd.DataFrame(encoder.transform(X_test[categorical_features]),
                                  columns=encoder.get_feature_names_out(categorical_features),
                                  index=X_test.index)
    X_train = X_train.drop(categorical_features, axis=1).join(X_train_encoded)
    X_test = X_test.drop(categorical_features, axis=1).join(X_test_encoded)

stage_message("Data Cleaning and Preprocessing Completed", color="#FF9800", icon="🧹")
print("   ➤ Dataset scaled using StandardScaler; no categorical encoding required.\n")

# =========================================
# Feature Engineering
# =========================================


X_train['Rooms_per_Household'] = X_train['AveRooms'] / X_train['AveOccup']
X_test['Rooms_per_Household'] = X_test['AveRooms'] / X_test['AveOccup']
X_train['Bedrooms_per_Room'] = X_train['AveBedrms'] / X_train['AveRooms']
X_test['Bedrooms_per_Room'] = X_test['AveBedrms'] / X_test['AveRooms']
X_train['Population_per_Household'] = X_train['Population'] / X_train['AveOccup']
X_test['Population_per_Household'] = X_test['Population'] / X_test['AveOccup']
#all_features = X_train.columns.tolist()

stage_message("Feature Engineering Completed", color="#009688", icon="🧩")
print("   ➤ Added derived features: Rooms_per_Household, Bedrooms_per_Room, Population_per_Household.\n")

# =========================================
# Exploratory Data Analysis (EDA)
# =========================================

sns.set(style="whitegrid")
palette = sns.color_palette("Set2")


# Histograms
housing_df['Rooms_per_Household'] = housing_df['AveRooms'] / housing_df['AveOccup']
housing_df['Bedrooms_per_Room'] = housing_df['AveBedrms'] / housing_df['AveRooms']
housing_df['Population_per_Household'] = housing_df['Population'] / housing_df['AveOccup']

derived_features = ['Rooms_per_Household', 'Bedrooms_per_Room', 'Population_per_Household']
all_features = numerical_features + derived_features
fig, axes = plt.subplots(4, 3, figsize=(18, 16))
axes = axes.flatten()

for i, feature in enumerate(all_features):
    sns.histplot(X_train[feature], bins=30, kde=True, ax=axes[i], color=palette[i % len(palette)])
    axes[i].set_title(f'Distribution of {feature}')
    axes[i].set_xlabel('')
    axes[i].set_ylabel('Count')

# Remove the last empty subplot (if features < 12)
if len(all_features) < len(axes):
    for j in range(len(all_features), len(axes)):
        fig.delaxes(axes[j])

plt.tight_layout()
plt.savefig("images/housing_feature_histograms.png")
plt.close()


# Scatter plots
plt.figure(figsize=(10,6))
sns.scatterplot(x=X_train['MedInc'], y=y_train, alpha=0.5, color='royalblue')
plt.title("Median Income vs Median House Value")
plt.xlabel("Median Income (scaled)")
plt.ylabel("Median House Value")
plt.tight_layout()
plt.savefig("images/median_income_vs_housevalue.png")
plt.close()

plt.figure(figsize=(10,6))
sns.scatterplot(x=X_train['Longitude'], y=X_train['Latitude'], hue=y_train, palette='viridis', alpha=0.5)
plt.title("Geographic Distribution of House Values")
plt.xlabel("Longitude")
plt.ylabel("Latitude")
plt.legend(title="MedHouseValue", bbox_to_anchor=(1.05,1), loc='upper left')
plt.tight_layout()
plt.savefig("images/geographic_distribution.png", dpi=300, bbox_inches='tight')
plt.close()

stage_message("Exploratory Data Analysis Completed — charts saved to /images/initial_report/", color="#00BCD4", icon="📊")
print("   ➤ Graphical outputs saved in: '/images/initial_report/'\n")

# =========================================
# Outlier Handling
# =========================================

def cap_outliers(df, features):
    capped = df.copy()
    for feature in features:
        Q1 = df[feature].quantile(0.25)
        Q3 = df[feature].quantile(0.75)
        IQR = Q3 - Q1
        lower = Q1 - 1.5*IQR
        upper = Q3 + 1.5*IQR
        capped[feature] = np.where(capped[feature] < lower, lower, capped[feature])
        capped[feature] = np.where(capped[feature] > upper, upper, capped[feature])
    return capped

X_train_capped = cap_outliers(X_train, all_features)
X_test_capped = cap_outliers(X_test, all_features)

stage_message("Outlier Handling Completed", color="#795548", icon="⚙️")
print("   ➤ Applied IQR-based capping method to mitigate extreme values.\n")

# =========================================
# Baseline Model(Median Predictor)
# =========================================
stage_message("Baseline Model Definition & Evaluation In Progress...", color="#8E24AA",icon="🧮")

baseline_pred = np.median(y_train)
baseline_mse = mean_squared_error(y_test, [baseline_pred]*len(y_test))
baseline_mae = mean_absolute_error(y_test, [baseline_pred]*len(y_test))
baseline_r2 = r2_score(y_test, [baseline_pred]*len(y_test))
#print("Baseline Model - MSE:", baseline_mse, "MAE:", baseline_mae, "R2:", baseline_r2)
baseline_results = pd.DataFrame({
    'Model': ['Baseline (Median Predictor)'],
    'MSE': [baseline_mse],
    'MAE': [baseline_mae],
    'R2': [baseline_r2]
})

# ===============================================
# 📊 Baseline Model Performance Summary (Enhanced)
# ===============================================
display(
    baseline_results.style.hide(axis='index')
    .set_table_styles([
        {'selector': 'th', 'props': [
            ('background-color', '#4C72B0'),
            ('color', 'white'),
            ('font-weight', 'bold'),
            ('text-align', 'center'),
            ('border', '1px solid #ddd')
        ]},
        {'selector': 'td', 'props': [
            ('text-align', 'center'),
            ('border', '1px solid #ddd'),
            ('font-weight', 'bold')
        ]}
    ])
    .format({'MSE': '{:.4f}', 'MAE': '{:.4f}', 'R2': '{:.4f}'})
    .set_caption("📊 Baseline Model – Performance Summary")
)

stage_message("Evaluating Initial Regression Models...",color="#1976D2",icon="📊")
print("   ➤ Models being evaluated: Linear Regression, Ridge Regression, Decision Tree, Random Forest, Gradient Boosting, KNN Regressor\n")

# ========================================
#  Regression Model Comparison (Initial Models)
# =========================================
# 
# After establishing a baseline (median predictor), this section introduces several 
# standard regression models — Linear Regression, Ridge, Decision Tree, Random Forest, 
# Gradient Boosting, and KNN — for initial performance evaluation. 
# These serve as the first machine learning benchmarks against which future 
# optimized and tuned models will be compared.

#
models = {
    "Linear Regression": LinearRegression(),
    "Ridge Regression": Ridge(alpha=1.0),
    "Decision Tree": DecisionTreeRegressor(random_state=42),
    "Random Forest": RandomForestRegressor(n_estimators=100, random_state=42),
    "Gradient Boosting": GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, random_state=42),
    "KNN Regressor": KNeighborsRegressor(n_neighbors=5)
}

results = []
for name, model in models.items():
    model.fit(X_train_capped, y_train)
    y_pred = model.predict(X_test_capped)
    results.append([name,
                    mean_squared_error(y_test, y_pred),
                    mean_absolute_error(y_test, y_pred),
                    r2_score(y_test, y_pred)])

results_df = pd.DataFrame(results, columns=['Model','MSE','MAE','R2'])
results_df.sort_values('R2', ascending=False, inplace=True)

# =====================================================
# 📈 Initial Regression Models Performance Summary
# =====================================================

display(
    results_df.style.hide(axis='index')
    .set_table_styles([
        {'selector': 'th', 'props': [
            ('background-color', '#43A047'),
            ('color', 'white'),
            ('font-weight', 'bold'),
            ('text-align', 'center'),
            ('border', '1px solid #ddd')
        ]},
        {'selector': 'td', 'props': [
            ('text-align', 'center'),
            ('border', '1px solid #ddd'),
            ('font-weight', 'bold')
        ]}
    ])
    .format({'MSE': '{:.4f}', 'MAE': '{:.4f}', 'R2': '{:.4f}'})
    .set_caption("📈 Initial Regression Models – Performance Summary")
)

# ==========================================================================
# Random Forest Regressor – Hyperparameter Optimization (RandomizedSearchCV)
# ==========================================================================

# RandomForest is a robust ensemble method that averages multiple decision trees to reduce overfitting.  
# However, exhaustive `GridSearchCV` on large datasets can be very time-consuming.  
# To optimize runtime without compromising accuracy, we switch to **`RandomizedSearchCV`**, which samples
# from a defined parameter space to find strong hyperparameters efficiently.


stage_message("Hyperparameter Optimization(RandomizedSearchCV on Random Forest) in Progress...",color="#2196F3",icon="🔍")

# Track runtime
start_time = time.time()

# ============================================================================================================
# Random Forest Regressor – Hyperparameter Optimization (RandomizedSearchCV) -Stage 1: Broad Randomized Search
# ============================================================================================================

param_dist_rf = {
    'n_estimators': [50, 100, 150, 200],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['sqrt', 'log2']
}

random_search_rf = RandomizedSearchCV(
    RandomForestRegressor(random_state=42),
    param_distributions=param_dist_rf,
    n_iter=6,                   
    cv=3,
    scoring='r2',
    n_jobs=-1,
    verbose=1,
    random_state=42
)
with contextlib.redirect_stdout(io.StringIO()):
    random_search_rf.fit(X_train_capped, y_train)
best_params_stage1 = random_search_rf.best_params_
rf_random_df = pd.DataFrame([best_params_stage1])
rf_random_df['Best CV R²'] = [f"{random_search_rf.best_score_:.4f}"]

display(
    rf_random_df.style.hide(axis='index')
    .set_table_styles([
        {'selector': 'th', 'props': [('background-color', '#1F77B4'),
                                     ('color', 'white'),
                                     ('font-weight', 'bold'),
                                     ('text-align', 'center')]},
        {'selector': 'td', 'props': [('text-align', 'center'),
                                     ('border', '1px solid #ddd')]}
    ])
    .set_caption("📊 Broad Randomized Search Results (Random Forest)")
)
stage_message( "Broad Randomized Search Completed Successfully – Optimal hyperparameter subset identified.",color="#4CAF50",icon="🔍") 

# ============================================================================================================
# Random Forest Regressor – Hyperparameter Optimization (RandomizedSearchCV) -Stage 3: Narrow Grid Search
# Only vary around best results — keep combinations <= 20
# ============================================================================================================

param_grid_rf = {
    'n_estimators': [best_params_stage1['n_estimators']],
    'max_depth': [best_params_stage1['max_depth'], None],
    'min_samples_split': [best_params_stage1['min_samples_split']],
    'min_samples_leaf': [best_params_stage1['min_samples_leaf']],
    'max_features': [best_params_stage1['max_features']]
}

grid_search_rf = GridSearchCV(
    RandomForestRegressor(random_state=42),
    param_grid=param_grid_rf,
    cv=3,
    scoring='r2',
    n_jobs=-1,
    verbose=0
)

grid_search_rf.fit(X_train_capped, y_train)
best_rf = grid_search_rf.best_estimator_
rf_grid_df = pd.DataFrame([grid_search_rf.best_params_])
rf_grid_df['Best CV R²'] = [f"{grid_search_rf.best_score_:.4f}"]

display(
    rf_grid_df.style.hide(axis='index')
    .set_table_styles([
        {'selector': 'th', 'props': [('background-color', '#2CA02C'),
                                     ('color', 'white'),
                                     ('font-weight', 'bold'),
                                     ('text-align', 'center')]},
        {'selector': 'td', 'props': [('text-align', 'center'),
                                     ('border', '1px solid #ddd')]}
    ])
    .set_caption("📈 Narrow Grid Search Results (Random Forest)")
)
stage_message("Narrow Grid Search Completed Successfully",color="#4CAF50",icon="🎯")

# =============================================
# Evaluate Final Tuned Model
# =============================================
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

y_pred_best_rf = best_rf.predict(X_test_capped)
train_time_min = (time.time() - start_time) / 60  # Convert seconds to minutes

# Compute metrics
rf_metrics = {
    'R² Score (Test)': [f"{r2_score(y_test, y_pred_best_rf):.4f}"],
    'MAE (Test)': [f"{mean_absolute_error(y_test, y_pred_best_rf):.4f}"],
    'MSE (Test)': [f"{mean_squared_error(y_test, y_pred_best_rf):.4f}"],
    'Training Time (min)': [f"{train_time_min:.2f}"]
}


# Create a single-row DataFrame for display
rf_final_df = pd.DataFrame(rf_metrics)

# Display formatted summary table
display(
    rf_final_df.style.hide(axis='index')
    .set_table_styles([
        {'selector': 'th', 'props': [('background-color', '#FFD700'),
                                     ('color', 'black'),
                                     ('font-weight', 'bold'),
                                     ('text-align', 'center')]},
        {'selector': 'td', 'props': [('text-align', 'center'),
                                     ('border', '1px solid #ddd'),
                                     ('font-weight', 'bold')]}
    ])
    .set_caption("🏆 Final Tuned Random Forest Performance Summary")
)

# ==============================================================================
#  Visualization 1: Final Random Forest (Optimized) - Actual vs Predicted Prices
# ==============================================================================

plt.figure(figsize=(8,6))
sns.scatterplot(x=y_test, y=y_pred_best_rf, alpha=0.6, color='royalblue')
plt.plot([y_test.min(), y_test.max()],
         [y_test.min(), y_test.max()],
         'r--', lw=2, label='Perfect Prediction')
plt.xlabel("Actual Median House Value")
plt.ylabel("Predicted Median House Value")
plt.title("Final Random Forest (Optimized) - Actual vs Predicted Prices")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.savefig("images/Random Forest (Optimized) - Actual vs Predicted Prices.png")
plt.close()

# ==================================================================
#  Visualization 2: Top 15 Feature Importances – Final Random Forest
# ==================================================================

feat_imp = (
    pd.DataFrame({
        'Feature': X_train_capped.columns,
        'Importance': best_rf.feature_importances_
    })
    .sort_values('Importance', ascending=False)
)

plt.figure(figsize=(10,6))
sns.barplot(x='Importance', y='Feature', hue='Feature',data=feat_imp.head(15), palette='bright')
plt.title("Top 15 Feature Importances – Final Random Forest")
plt.xlabel("Importance Score")
plt.ylabel("Feature")
plt.tight_layout()
plt.savefig("images/Top 15 Feature Importances.png")
plt.close()

# ================================================================================
# Additional Models + Optimized SVR Tuning + Bias–Variance Analysis
# Includes: SGDRegressor, SVR with kernel trick, randomized hyper-parameter search,
# bias–variance learning curves, and epsilon-tube visualization.
# ================================================================================

# ==========================
#   SGD Regressor (Baseline)
# ==========================

stage_message("Training SGD Regressor...",color="#F57C00",icon="⚙️")
sgd = SGDRegressor(max_iter=1000, tol=1e-3, random_state=42)
sgd.fit(X_train_capped, y_train)
y_pred_sgd = sgd.predict(X_test_capped)
sgd_metrics = {
    'R² Score': [f"{r2_score(y_test, y_pred_sgd):.4f}"],
    'MAE': [f"{mean_absolute_error(y_test, y_pred_sgd):.4f}"],
    'MSE': [f"{mean_squared_error(y_test, y_pred_sgd):.4f}"]
}
sgd_df = pd.DataFrame(sgd_metrics)

display(
    sgd_df.style.hide(axis='index')
    .set_table_styles([
        {'selector': 'th', 'props': [('background-color', '#FF7F0E'),
                                     ('color', 'white'),
                                     ('font-weight', 'bold'),
                                     ('text-align', 'center')]},
        {'selector': 'td', 'props': [('text-align', 'center'),
                                     ('border', '1px solid #ddd'),
                                     ('font-weight', 'bold')]}
    ])
    .set_caption("⚙️ SGD Regressor – Performance Summary")
)

# ===================================================================
#   Baseline SVR with Kernel Trick (with subsample to reduce runtime)
# ===================================================================
stage_message("Training Baseline SVR (RBF Kernel) on 15% Data for Efficiency...",color="#7B1FA2",icon="⚙️")
sample_frac = 0.15
X_train_svr = X_train_capped.sample(frac=sample_frac, random_state=42)
y_train_svr = y_train.loc[X_train_svr.index]
X_test_svr = X_test_capped.sample(frac=sample_frac, random_state=42)
y_test_svr = y_test.loc[X_test_svr.index]

svr = SVR(kernel='rbf', C=100, gamma=0.1, epsilon=0.1)
svr.fit(X_train_svr, y_train_svr)
y_pred_svr = svr.predict(X_test_svr)

svr_base_metrics = {
    'R² Score': [f"{r2_score(y_test_svr, y_pred_svr):.4f}"],
    'MAE': [f"{mean_absolute_error(y_test_svr, y_pred_svr):.4f}"],
    'MSE': [f"{mean_squared_error(y_test_svr, y_pred_svr):.4f}"]
}
svr_base_df = pd.DataFrame(svr_base_metrics)

display(
    svr_base_df.style.hide(axis='index')
    .set_table_styles([
        {'selector': 'th', 'props': [('background-color', '#9467BD'),
                                     ('color', 'white'),
                                     ('font-weight', 'bold'),
                                     ('text-align', 'center')]},
        {'selector': 'td', 'props': [('text-align', 'center'),
                                     ('border', '1px solid #ddd'),
                                     ('font-weight', 'bold')]}
    ])
    .set_caption("📉 Baseline SVR (Subsampled) – Performance Summary")
)

# ======================================================
# RandomizedSearchCV for SVR Hyperparameter Optimization
# ======================================================
stage_message("Randomized Search CV: Optimizing SVR Hyperparameters...",color="#0288D1",icon="🧠")
param_dist_svr = {
    'C': np.logspace(1, 3, 5),         # [10, 31.6, 100, 316, 1000]
    'gamma': [0.01, 0.05, 0.1, 0.2],
    'epsilon': [0.01, 0.05, 0.1, 0.2]
}

random_search_svr = RandomizedSearchCV(
    SVR(kernel='rbf'),
    param_distributions=param_dist_svr,
    n_iter=8,                # small and fast
    cv=3,
    scoring='r2',
    n_jobs=-1,
    verbose=0,
    random_state=42
)

random_search_svr.fit(X_train_svr, y_train_svr)
best_svr = random_search_svr.best_estimator_
svr_tuned_df = pd.DataFrame([random_search_svr.best_params_])
svr_tuned_df['Best CV R²'] = [f"{random_search_svr.best_score_:.4f}"]

display(
    svr_tuned_df.style.hide(axis='index')
    .set_table_styles([
        {'selector': 'th', 'props': [('background-color', '#17BECF'),
                                     ('color', 'white'),
                                     ('font-weight', 'bold'),
                                     ('text-align', 'center')]},
        {'selector': 'td', 'props': [('text-align', 'center'),
                                     ('border', '1px solid #ddd')]}
    ])
    .set_caption("✅ SVR Randomized Search – Best Parameters & CV Results")
)

# Evaluate tuned SVR
y_pred_best_svr = best_svr.predict(X_test_svr)
svr_final_metrics = {
    'R² Score (Test)': [f"{r2_score(y_test_svr, y_pred_best_svr):.4f}"]
}

svr_final_df = pd.DataFrame(svr_final_metrics)

display(
    svr_final_df.style.hide(axis='index')
    .set_table_styles([
        {'selector': 'th', 'props': [
            ('background-color', '#9C27B0'),
            ('color', 'white'),
            ('font-weight', 'bold'),
            ('text-align', 'center'),
            ('border', '1px solid #ddd')
        ]},
        {'selector': 'td', 'props': [
            ('text-align', 'center'),
            ('border', '1px solid #ddd'),
            ('font-weight', 'bold')
        ]}
    ])
    .set_caption("🏆 Tuned SVR Evaluation – Final Test Performance")
)

# ===============================
# Bias–Variance (Learning Curves)
# ===============================
stage_message("Generating Learning Curves (Bias–Variance Analysis)...",color="#009688",icon="📈")

def plot_learning_curve(model, X, y, title):
    train_sizes, train_scores, test_scores = learning_curve(
        model, X, y, cv=3, scoring='r2', n_jobs=-1,
        train_sizes=np.linspace(0.1, 1.0, 5)
    )
    plt.figure()
    plt.plot(train_sizes, np.mean(train_scores, axis=1), 'o-', color='blue', label='Train R²')
    plt.plot(train_sizes, np.mean(test_scores, axis=1), 'o-', color='green', label='CV R²')
    plt.xlabel("Training Examples")
    plt.ylabel("R² Score")
    plt.title(title)
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(f"images/{title}.png")
    plt.close()

plot_learning_curve(sgd, X_train_capped, y_train, title="Learning Curve – SGD Regressor")
plot_learning_curve(best_svr, X_train_svr, y_train_svr, title="Learning Curve – Tuned SVR (Subsampled)")


# ======================================================
# SVR Margin (Epsilon-Tube) / Maximum Margin Concept
# ======================================================
stage_message("Visualizing SVR Margin (Epsilon-Tube)...",color="#795548",icon="📊")

plt.figure(figsize=(8,5))
plt.scatter(X_train_svr['MedInc'], y_train_svr, alpha=0.3, label='Training Points')

# Create 500 evenly spaced values of MedInc
X_plot = np.linspace(X_train_svr['MedInc'].min(), X_train_svr['MedInc'].max(), 500)

# Create DataFrame with same columns as training, other features filled with median
X_plot_df = pd.DataFrame(np.tile(X_train_svr.median().values, (500, 1)), columns=X_train_svr.columns)
X_plot_df['MedInc'] = X_plot

# Predict using SVR
y_svr_plot = best_svr.predict(X_plot_df)

# Visualization - SVR Prediction
plt.plot(X_plot, y_svr_plot, color='red', label='SVR Prediction')
plt.fill_between(
    X_plot,
    y_svr_plot - best_svr.epsilon,
    y_svr_plot + best_svr.epsilon,
    color='orange', alpha=0.3, label='Epsilon Tube (Margin)'
)
plt.xlabel("Median Income (Scaled)")
plt.ylabel("Median House Value")
plt.title("SVR Epsilon Tube – Maximum Margin Concept (Subsampled)")
plt.legend()
plt.tight_layout()
plt.savefig("images/SVR-Episilon Tube_Maximum_margin_concept.png")
plt.close()

# ================
# Model Comparison 
# ================
stage_message("Generating Full Model Comparison with Visuals...",color="#607D8B",icon="📊")
# Recalculate predictions for key models
y_pred_best_svr_full = best_svr.predict(X_test_capped)
y_pred_best_rf = best_rf.predict(X_test_capped)

# Prepare comparison data (exclude SGD initially)
comparison_data_main = [
    ["Linear Regression", mean_squared_error(y_test, models["Linear Regression"].predict(X_test_capped)),
     mean_absolute_error(y_test, models["Linear Regression"].predict(X_test_capped)),
     r2_score(y_test, models["Linear Regression"].predict(X_test_capped))],
    
    ["Ridge Regression", mean_squared_error(y_test, models["Ridge Regression"].predict(X_test_capped)),
     mean_absolute_error(y_test, models["Ridge Regression"].predict(X_test_capped)),
     r2_score(y_test, models["Ridge Regression"].predict(X_test_capped))],
    
    ["Decision Tree", mean_squared_error(y_test, models["Decision Tree"].predict(X_test_capped)),
     mean_absolute_error(y_test, models["Decision Tree"].predict(X_test_capped)),
     r2_score(y_test, models["Decision Tree"].predict(X_test_capped))],
    
    ["Random Forest (Tuned)", mean_squared_error(y_test, y_pred_best_rf),
     mean_absolute_error(y_test, y_pred_best_rf),
     r2_score(y_test, y_pred_best_rf)],
    
    ["Gradient Boosting", mean_squared_error(y_test, models["Gradient Boosting"].predict(X_test_capped)),
     mean_absolute_error(y_test, models["Gradient Boosting"].predict(X_test_capped)),
     r2_score(y_test, models["Gradient Boosting"].predict(X_test_capped))],
    
    ["KNN Regressor", mean_squared_error(y_test, models["KNN Regressor"].predict(X_test_capped)),
     mean_absolute_error(y_test, models["KNN Regressor"].predict(X_test_capped)),
     r2_score(y_test, models["KNN Regressor"].predict(X_test_capped))],
    
    ["Tuned SVR", mean_squared_error(y_test, y_pred_best_svr_full),
     mean_absolute_error(y_test, y_pred_best_svr_full),
     r2_score(y_test, y_pred_best_svr_full)]
]

# Stochastic Gradient Descent (SGD) Regressor (extreme scale)
comparison_sgd = [
    ["SGD Regressor", mean_squared_error(y_test, y_pred_sgd),
     mean_absolute_error(y_test, y_pred_sgd),
     r2_score(y_test, y_pred_sgd)]
]

# Convert to DataFrames
df_main = pd.DataFrame(comparison_data_main, columns=['Model','MSE','MAE','R2']).sort_values('R2', ascending=False).reset_index(drop=True)
df_sgd = pd.DataFrame(comparison_sgd, columns=['Model','MSE','MAE','R2'])

all_results_full_sorted = pd.concat([df_main, df_sgd]).sort_values('R2', ascending=False).reset_index(drop=True)

# ============================================================
# 🏁 Final Model Comparison – Comprehensive Performance Summary
# ============================================================

display(
    all_results_full_sorted.style.hide(axis='index')
    .set_table_styles([
        {'selector': 'th', 'props': [
            ('background-color', '#1976D2'),
            ('color', 'white'),
            ('font-weight', 'bold'),
            ('text-align', 'center'),
            ('border', '1px solid #ddd')
        ]},
        {'selector': 'td', 'props': [
            ('text-align', 'center'),
            ('border', '1px solid #ddd'),
            ('font-weight', 'bold')
        ]}
    ])
    .format({'MSE': '{:.4f}', 'MAE': '{:.4f}', 'R2': '{:.4f}'})
    .set_caption("🏁 Final Model Comparison – Comprehensive Performance Summary")
)

display(HTML("""
<div style='background-color:#FFF3CD; color:#856404; border-left:8px solid #FF9800;
            padding:12px 16px; margin:10px 0; border-radius:6px;'>
    <p style='font-size:16px; font-weight:bold; margin:0;'>
        ⚠️ <span style='font-size:18px;'>SGD Regressor Instability Alert</span>
    </p>
    <p style='margin:6px 0 0 28px; font-size:14px;'>
        The SGD Regressor is highly unstable on this dataset — its performance metrics are on a vastly different scale, 
        indicating potential convergence or scaling issues.
    </p>
</div>
"""))

# Normalize metrics for visualization (excluding SGD)
df_main_norm = df_main.copy()
df_main_norm['MSE_norm'] = df_main_norm['MSE'] / df_main_norm['MSE'].max()
df_main_norm['MAE_norm'] = df_main_norm['MAE'] / df_main_norm['MAE'].max()
df_main_norm['R2_norm'] = df_main_norm['R2']  # R2 already 0–1

# custom color palette
custom_palette = list(mcolors.TABLEAU_COLORS.values())[:len(df_main_norm)]

# ===============================================================
# Model Comparison – R² Score (Normalized) - Horizontal bar plots
# ===============================================================

plt.figure(figsize=(10,6))
sns.barplot(x='R2_norm', y='Model', hue='Model',data=df_main_norm, palette=custom_palette)
plt.title("Model Comparison – R² Score (Normalized)")
plt.xlabel("R² Score")
plt.ylabel("Model")
plt.xlim(0,1)
plt.tight_layout()
plt.savefig("images/Model Comparison - Normalized - R2 score.png")
plt.close()


# ===============================================================
# Model Comparison – MAE (Normalized) - Horizontal bar plots
# ===============================================================
plt.figure(figsize=(10,6))
sns.barplot(x='MAE_norm', y='Model',hue='Model', data=df_main_norm, palette=custom_palette)
plt.title("Model Comparison – MAE (Normalized)")
plt.xlabel("Normalized MAE")
plt.ylabel("Model")
plt.tight_layout()
plt.savefig("images/Model Comparison - Normalized - MAE.png")
plt.close()


# ===============================================================
# Model Comparison – MSE (Normalized) - Horizontal bar plots
# ===============================================================
plt.figure(figsize=(10,6))
sns.barplot(x='MSE_norm', y='Model',hue='Model', data=df_main_norm, palette=custom_palette)
plt.title("Model Comparison – MSE (Normalized)")
plt.xlabel("Normalized MSE")
plt.ylabel("Model")
plt.tight_layout()
plt.savefig("images/Model Comparison - Normalized - MSE.png")
plt.close()

# =========================================
# Model Evaluation and best model selection
# =========================================
best_r2_model = df_main.loc[df_main['R2'].idxmax()]['Model']
best_r2 = df_main['R2'].max()

best_mae_model = df_main.loc[df_main['MAE'].idxmin()]['Model']
best_mae = df_main['MAE'].min()

best_mse_model = df_main.loc[df_main['MSE'].idxmin()]['Model']
best_mse = df_main['MSE'].min()

# =========================================
# Final Summary & Recommendations
# =========================================
display(HTML(f"""
<div style='background-color:#F5F7FA; border-left:8px solid #4CAF50;
            padding:16px 20px; margin:15px 0; border-radius:8px;
            box-shadow:0 2px 4px rgba(0,0,0,0.05);'>

    <h3 style='color:#2E7D32; font-weight:bold; margin-top:0;'>
        📌 Final Summary & Recommendations
    </h3>

    <table style='width:100%; border-collapse:collapse; font-size:14px; margin-top:10px;'>
        <tr style='background-color:#E8F5E9;'>
            <th style='text-align:middle; padding:8px; color:#1B5E20;'>Metric</th>
            <th style='text-align:middle; padding:8px; color:#1B5E20;'>Model</th>
            <th style='text-align:middle; padding:8px; color:#1B5E20;'>Score</th>
        </tr>
        <tr>
            <td style='padding:8px;'>Highest R² (Best Overall Fit)</td>
            <td style='padding:8px; font-weight:bold;'>{best_r2_model}</td>
            <td style='padding:8px; font-weight:bold;'>{best_r2:.3f}</td>
        </tr>
        <tr>
            <td style='padding:8px;'>Lowest MAE (Best Prediction Accuracy)</td>
            <td style='padding:8px; font-weight:bold;'>{best_mae_model}</td>
            <td style='padding:8px; font-weight:bold;'>{best_mae:.3f}</td>
        </tr>
        <tr>
            <td style='padding:8px;'>Lowest MSE (Best Error Minimization)</td>
            <td style='padding:8px; font-weight:bold;'>{best_mse_model}</td>
            <td style='padding:8px; font-weight:bold;'>{best_mse:.3f}</td>
        </tr>
    </table>

    <div style='margin-top:15px; line-height:1.6; color:#333; font-size:14px;'>
        <p>✅ <b>Observation:</b> Random Forest and Gradient Boosting emerge as strong ensemble learners, 
        demonstrating superior balance between bias and variance.</p>
        <p>⚙️ <b>Tuned SVR:</b> Provides a competitive non-linear alternative capable of modeling complex relationships.</p>
        <p>⚠️ <b>SGD Regressor:</b> Exhibits extreme instability, producing divergent metrics; it is <b>not recommended</b> for this dataset.</p>
        <p>📈 <b>Recommendation:</b> The <b>{best_r2_model}</b> model is recommended for production deployment or further refinement,
        supported by its excellent predictive performance and interpretability.</p>
    </div>
</div>
"""))

Numerical Features: ['MedInc', 'HouseAge', 'AveRooms', 'AveBedrms', 'Population', 'AveOccup', 'Latitude', 'Longitude']
Categorical Features: []


   ➤ Feature types identified: Numerical and Derived ratios analyzed.



   ➤ Dataset scaled using StandardScaler; no categorical encoding required.



   ➤ Added derived features: Rooms_per_Household, Bedrooms_per_Room, Population_per_Household.



   ➤ Graphical outputs saved in: '/images/initial_report/'



...

R² Score,MAE,MSE
0.3821,0.6584,0.7549


gamma,epsilon,C,Best CV R²
0.010000,0.200000,31.622777,0.6178


R² Score (Test)
0.6484


Model,MSE,MAE,R2
Random Forest (Tuned),0.2722,0.3514,0.7923
Gradient Boosting,0.2989,0.3772,0.7719
Ridge Regression,0.4675,0.5048,0.6432
Linear Regression,0.4675,0.5051,0.6432
Tuned SVR,0.4845,0.5006,0.6303
Decision Tree,0.5281,0.4775,0.5970
KNN Regressor,0.6741,0.6192,0.4855
SGD Regressor,95538.3229,245.0493,-72906.2561


Metric,Model,Score
Highest R² (Best Overall Fit),Random Forest (Tuned),0.792
Lowest MAE (Best Prediction Accuracy),Random Forest (Tuned),0.351
Lowest MSE (Best Error Minimization),Random Forest (Tuned),0.272
